In [20]:
import json
import numpy as np
import pandas as pd
from glob import glob
from sklearn.metrics import classification_report, f1_score

In [21]:
strats = ["igcngru_features", "idarkvec", "features"]
DATA_SOURCE = "darknet"

In [22]:
data_dir = f"../data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/test"
days = [ f.split('/')[-1].split('_')[1] for f in glob(f"{data_dir}/idarkvec_*_fold00.csv")]
days.sort()
days

['20221021',
 '20221022',
 '20221023',
 '20221024',
 '20221025',
 '20221026',
 '20221027',
 '20221028',
 '20221029',
 '20221030',
 '20221031']

In [23]:
df = pd.read_csv(f"../data/2022/input/stacking_predictions/out/k3/{DATA_SOURCE}/test/idarkvec_20221021_fold00.csv")

In [24]:
df = df[df.y_true != "unknown"].copy()

In [25]:
df.head()

,Unnamed: 0,src_ip,censys,driftnet,internetcensus,intrinsec,ipip,mirai,onyphe,rapid7,securitytrails,shadowserver,shodan,u_mich,unk_bruteforcer,unk_exploiter,unk_spammer,unknown,y_true
0,12982,167.94.138.102,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,censys
1,12983,167.94.138.146,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,censys
2,12984,118.40.8.149,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,mirai
3,12985,103.126.245.10,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,mirai
4,12986,193.142.146.35,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,unk_bruteforcer


In [26]:
prob_cols = [
    "censys",
    "driftnet",
    "internetcensus",
    "intrinsec",
    "ipip",
    "mirai",
    "onyphe",
    "rapid7",
    "securitytrails",
    "shadowserver",
    "shodan",
    "u_mich",
    "unk_bruteforcer",
    "unk_exploiter",
    "unk_spammer",
    "unknown"
]
prob_cols.sort()

In [27]:
metrics = {}
for strat in strats:
    metrics[strat] = {}
    for day in days:
        y, preds = [], []
        for fold in np.arange(10):
            df = pd.read_csv(f"{data_dir}/{strat}_{day}_fold0{fold}.csv")
            ps = df[prob_cols].values.argmax(axis=1)
            preds.append([prob_cols[c] for c in ps])
            y.append(df.y_true.values)

        preds = np.hstack(preds)
        y = np.hstack(y)
        report = classification_report(
            y,
            preds,
            labels=np.unique(y),
            output_dict=True,
            zero_division=0,
        )
        for c in prob_cols + ["macro avg"]:
            if c in report:
                if c not in metrics[strat]:
                    metrics[strat][c] = []
                metrics[strat][c].append(report[c]["f1-score"])

In [28]:
scores = pd.DataFrame(metrics)
scores

,igcngru_features,idarkvec,features
censys,"[0.7486631016042781, 0.7650273224043717, 0.787...","[0.7126436781609196, 0.6627218934911243, 0.749...","[0.911764705882353, 0.9674418604651163, 0.9626..."
driftnet,"[0.97265625, 0.9764705882352942, 0.97821782178...","[0.9863013698630138, 0.9901768172888015, 0.984...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99604743..."
internetcensus,"[0.9134199134199132, 0.9192825112107623, 0.909...","[0.9653679653679653, 0.9727272727272728, 0.977...","[0.9259259259259259, 0.9716981132075471, 0.966..."
intrinsec,"[0.9565217391304348, 0.5714285714285715, 0.5, ...","[1.0, 0.6956521739130435, 0.923076923076923, 0...","[0.13333333333333333, 0.7058823529411764, 0.5,..."
ipip,"[1.0, 1.0, 0.9600000000000001, 1.0, 0.96000000...","[0.15384615384615385, 0.15384615384615385, 0.1...","[1.0, 1.0, 0.9090909090909091, 1.0, 1.0, 1.0, ..."
mirai,"[0.9965078573210276, 0.9971156257837974, 0.995...","[0.9908818386210342, 0.9936050156739812, 0.992...","[0.998006230529595, 0.9983085886111633, 0.9978..."
onyphe,"[0.9101123595505618, 0.8977272727272727, 0.928...","[0.9223300970873787, 0.926829268292683, 0.9696...","[0.9735449735449735, 0.9162011173184357, 0.951..."
securitytrails,"[0.9473684210526316, 0.9473684210526316, 0.9, ...","[0.972972972972973, 1.0, 1.0, 1.0, 1.0, 1.0, 1...","[0.9473684210526316, 0.9473684210526316, 1.0, ..."
shadowserver,"[0.6527777777777778, 0.665158371040724, 0.6438...","[0.9859649122807017, 0.9912739965095986, 0.996...","[0.6681818181818182, 0.6742081447963801, 0.662..."
shodan,"[0.55, 0.5263157894736842, 0.6976744186046512,...","[0.7727272727272727, 0.7555555555555555, 0.775...","[0.8846153846153846, 0.8461538461538461, 0.892..."


In [29]:
def mean(values):
    return np.trunc(np.std(values) * 100) / 100

result_df = scores.applymap(mean)
macro = result_df.sort_index()
macro

,igcngru_features,idarkvec,features
censys,0.03,0.04,0.02
driftnet,0.00,0.00,0.00
internetcensus,0.03,0.00,0.02
intrinsec,0.16,0.10,0.18
ipip,0.02,0.07,0.03
macro avg,0.02,0.02,0.03
mirai,0.00,0.00,0.00
onyphe,0.03,0.02,0.02
rapid7,0.01,0.00,0.00
securitytrails,0.02,0.00,0.04


In [30]:
for col in prob_cols:
    if col in macro.index:
        print(";".join([str(v) for v in macro.loc[col].values]))
    else:
        print("0.0;0.0;0.0")

0.03;0.04;0.02
0.0;0.0;0.0
0.03;0.0;0.02
0.16;0.1;0.18
0.02;0.07;0.03
0.0;0.0;0.0
0.03;0.02;0.02
0.01;0.0;0.0
0.02;0.0;0.04
0.02;0.0;0.02
0.1;0.06;0.03
0.02;0.02;0.04
0.04;0.05;0.04
0.17;0.25;0.2
0.01;0.02;0.02
0.0;0.0;0.0


In [31]:
def stdev(values):
    return np.trunc(np.std(values) * 100) / 100

result_df = scores.applymap(stdev)
macro = result_df.sort_index()
macro

,igcngru_features,idarkvec,features
censys,0.03,0.04,0.02
driftnet,0.00,0.00,0.00
internetcensus,0.03,0.00,0.02
intrinsec,0.16,0.10,0.18
ipip,0.02,0.07,0.03
macro avg,0.02,0.02,0.03
mirai,0.00,0.00,0.00
onyphe,0.03,0.02,0.02
rapid7,0.01,0.00,0.00
securitytrails,0.02,0.00,0.04


In [32]:
for col in prob_cols:
    if col in macro.index:
        print(";".join([str(v) for v in macro.loc[col].values]))
    else:
        print("0.0;0.0;0.0")

0.03;0.04;0.02
0.0;0.0;0.0
0.03;0.0;0.02
0.16;0.1;0.18
0.02;0.07;0.03
0.0;0.0;0.0
0.03;0.02;0.02
0.01;0.0;0.0
0.02;0.0;0.04
0.02;0.0;0.02
0.1;0.06;0.03
0.02;0.02;0.04
0.04;0.05;0.04
0.17;0.25;0.2
0.01;0.02;0.02
0.0;0.0;0.0
